<a href="https://colab.research.google.com/github/Alexandr20i/BigData/blob/main/BigData_LR0_Belov_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.8294082019291567),
 (1, 1.8294082019291567),
 (2, 1.8294082019291567),
 (3, 1.8294082019291567),
 (4, 1.8294082019291567)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('banana', 2), ('what', 10)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.012265999527011129),
   (None, 0.032351340486612146),
   (None, 0.03846712479414216),
   (None, 0.052032613873847944),
   (None, 0.15773407339644951),
   (None, 0.20331208032690118),
   (None, 0.21891705553651097),
   (None, 0.253163563605703),
   (None, 0.26635500496748465),
   (None, 0.3432510158145392),
   (None, 0.36815931453270667),
   (None, 0.39065500667086817),
   (None, 0.417178675522929)]),
 (1,
  [(None, 0.5090829994374483),
   (None, 0.5870367118165211),
   (None, 0.6169764889704248),
   (None, 0.6593726735238871),
   (None, 0.6652279650111954),
   (None, 0.7073106120645201),
   (None, 0.7166834838979158),
   (None, 0.7695680290250446),
   (None, 0.7779429098815568),
   (None, 0.8298366495444975),
   (None, 0.8652592641206167),
   (None, 0.8870505649007816),
   (None, 0.8991827030005124),
   (None, 0.899762325614491),
   (None, 0.9283845491476055),
   (None, 0.9785550393754211),
   (None, 0.9810450383853389)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
import random

def MAP(list_num): return max(list_num)

def REDUCE(list_num): return max(list_num)

def RECORDREADER(count): return [random.randint(0, 100) for i in range(count)]

record = RECORDREADER(5)


parts = 5
record_partitional = [record[d:d + parts] for d in range(0, len(record), parts)]
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))
print(record_partitional)

57
[[57, 42, 39, 16, 37]]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
from typing import Iterator
import random

def RECORDREADER(count):
    return [random.randint(0, 50) for i in range(count)]

def MAP(num):
    return (1, num)

def REDUCE(_, numbers:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for number in numbers:
      sum+=number
      count+=1
    yield('AVG', sum/count)

map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(output)
print(map_output)


[('AVG', 32.6)]
[(1, 4), (1, 32), (1, 34), (1, 50), (1, 45), (1, 12), (1, 38), (1, 34), (1, 28), (1, 49)]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(_, numbers:Iterator[NamedTuple]):
    sum = 0
    count = 0
    for number in numbers:
      sum+=number
      count+=1
    yield('AVG', sum/count)


def RECORDREADER(count):
    return [random.randint(0, 5) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(3)))
shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print(shuffle_output)

output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(output)

[(1, [0, 4, 5])]
[('AVG', 3.0)]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'is', 'it']), (1, ['a', 'banana', 'what'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
from collections import defaultdict
from typing import Iterator, NamedTuple

def map_select(_: None, row: NamedTuple) -> Iterator[tuple[NamedTuple, NamedTuple]]:# Функция Map для выбора женщин.
    if row.gender == "female":
        yield (row, row)

def reduce_select(row: NamedTuple, rows: Iterator[NamedTuple]) -> Iterator[tuple[NamedTuple, list[NamedTuple]]]:# Функция Reduce для группировки женщин по характеристике (row).
    yield (row, list(rows))  # Преобразуем итератор в список

def record_reader() -> list[tuple[int, NamedTuple]]: # Функция чтения записей.
    return [(u.id, u) for u in input_collection]  # предполагается, что input_collection определен где-то выше

def group_by_key(iterable: list[tuple[NamedTuple, NamedTuple]]) -> list[tuple[NamedTuple, list[NamedTuple]]]: # Группировка по ключу (Shuffle).
    grouped_data = defaultdict(list)
    for key, value in iterable:
        grouped_data[key].append(value)
    return list(grouped_data.items())

def flatten(list_of_lists: list[list]) -> list: # Функция для "расплющивания" списка списков.
    return [item for sublist in list_of_lists for item in sublist]

def map_reduce(data: list[tuple[int, NamedTuple]]) -> list[tuple[NamedTuple, list[NamedTuple]]]: # Функция MapReduce.
    map_output = list(flatten(map(lambda x: list(map_select(None, x[1])), data))) # Применяем map_select к каждому элементу
    shuffle_output = group_by_key(map_output)
    reduce_output = list(flatten(map(lambda x: list(reduce_select(x[0], iter(x[1]))), shuffle_output)))
    return reduce_output


output = map_reduce(record_reader())
print(output)

[(User(id=1, age=25, social_contacts=240, gender='female'), [User(id=1, age=25, social_contacts=240, gender='female')]), (User(id=2, age=25, social_contacts=500, gender='female'), [User(id=2, age=25, social_contacts=500, gender='female')]), (User(id=3, age=33, social_contacts=800, gender='female'), [User(id=3, age=33, social_contacts=800, gender='female')])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
input_collection_2 = [
    dict(id=0, age=51, gender="male", social_contacts=21),
    dict(id=0, age=22, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="undefined", social_contacts=632),
    dict(id=2, age=11, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="undefined", social_contacts=27),
    dict(id=3, age=41, gender="male", social_contacts=621),
]
def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        # Удаляем поле gender у полученого словаря
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)


def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u["id"], u) for u in input_collection_2]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[(0,
  [{'id': 0, 'age': 51, 'gender': 'male', 'social_contacts': 21},
   {'id': 0, 'age': 22, 'gender': 'female', 'social_contacts': 240}]),
 (1, [{'id': 1, 'age': 27, 'social_contacts': 632}]),
 (2,
  [{'id': 2, 'age': 11, 'gender': 'male', 'social_contacts': 123},
   {'id': 2, 'age': 35, 'social_contacts': 27}]),
 (3, [{'id': 3, 'age': 41, 'gender': 'male', 'social_contacts': 621}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=125),
    User(id=2, age=21, gender="female", social_contacts=123),
    User(id=3, age=21, gender="male", social_contacts=63),
    User(id=4, age=19, gender="male", social_contacts=725),
    User(id=5, age=38, gender="female", social_contacts=242),
]

input_collection_b = [
    User(id=5, age=32, gender="female", social_contacts=213),
    User(id=6, age=76, gender="male", social_contacts=4679),
    User(id=7, age=44, gender="female", social_contacts=12),
]

# Группировка пользователей по id
def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)


# Выходное значение равно (t, t) независимо от присутствия пользователя в только одной или обоих выборках
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=1, age=54, social_contacts=125, gender='male'),
  User(id=1, age=54, social_contacts=125, gender='male')),
 (User(id=2, age=21, social_contacts=123, gender='female'),
  User(id=2, age=21, social_contacts=123, gender='female')),
 (User(id=3, age=21, social_contacts=63, gender='male'),
  User(id=3, age=21, social_contacts=63, gender='male')),
 (User(id=4, age=19, social_contacts=725, gender='male'),
  User(id=4, age=19, social_contacts=725, gender='male')),
 (User(id=5, age=38, social_contacts=242, gender='female'),
  User(id=5, age=38, social_contacts=242, gender='female')),
 (User(id=6, age=76, social_contacts=4679, gender='male'),
  User(id=6, age=76, social_contacts=4679, gender='male')),
 (User(id=7, age=44, social_contacts=12, gender='female'),
  User(id=7, age=44, social_contacts=12, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)


def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[User(id=5, age=38, social_contacts=242, gender='female'),
  User(id=5, age=32, social_contacts=213, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
# Группировка по пользователям
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)


# Возвращает пользователей, входящих только в первую выборку (и не входящих во вторую)
def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)


# На первом месте стоит номер выборки
def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]


output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=1, age=54, social_contacts=125, gender='male'),
 User(id=2, age=21, social_contacts=123, gender='female'),
 User(id=3, age=21, social_contacts=63, gender='male'),
 User(id=4, age=19, social_contacts=725, gender='male'),
 User(id=5, age=38, social_contacts=242, gender='female')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    city_id: int


users_collection = [
    User(id=1, age=53, gender="male", social_contacts=34, city_id=1),
    User(id=2, age=23, gender="female", social_contacts=22, city_id=2),
    User(id=3, age=19, gender="male", social_contacts=234, city_id=2),
    User(id=4, age=12, gender="male", social_contacts=764, city_id=3),
    User(id=5, age=65, gender="female", social_contacts=34, city_id=4),
]


class City(NamedTuple):
    id: int
    name: str


cities_collection = [
    City(id=1, name="Samara"),
    City(id=2, name="Moscow"),
    City(id=3, name="Samara"),
    City(id=4, name="SUUIIzran"),
    City(id=5, name="SPB"),
]

def MAP_JOIN(city_id, row):
    yield (city_id, row)


def REDUCE_JOIN(city_id, rows):
    users = []
    city = None

    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            city = row

    for row in rows:
        if type(row) is User:
            yield (row, row.city_id, city)


def RECORDREADER():
    return [(user.city_id, user) for user in users_collection] + [
        (city.id, city) for city in cities_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=53, social_contacts=34, gender='male', city_id=1),
  1,
  City(id=1, name='Samara')),
 (User(id=2, age=23, social_contacts=22, gender='female', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=3, age=19, social_contacts=234, gender='male', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=4, age=12, social_contacts=764, gender='male', city_id=3),
  3,
  City(id=3, name='Samara')),
 (User(id=5, age=65, social_contacts=34, gender='female', city_id=4),
  4,
  City(id=4, name='SUUIIzran'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
def MAP_GROUP(city_id, user, city):
    yield (city_id, user)


def REDUCE_GROUP(city_id, rows):
    yield f"city with id={city_id} has {len(rows)} user(s)"


def RECORDREADER():
    return [(city_id, user, city) for user, city_id, city in joined_data]


output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output


['city with id=1 has 1 user(s)',
 'city with id=2 has 2 user(s)',
 'city with id=3 has 1 user(s)',
 'city with id=4 has 1 user(s)']

#

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
import numpy as np

# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(x[0], x[1]), RECORDREADER())))))


Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np

I = 2
J = 3
K = 2 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)


def record_reader():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])


def mapper(k1, v1):
    j, k = k1
    w = v1

    for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)


def reducer(key, values):
    i, k = key

    k3 = (i, k)

    v3 = sum(values)  # Используем встроенную функцию sum()

    yield (k3, v3)


def group_by_key(items):
    groups = {}
    for key, value in items:
        if key not in groups:
            groups[key] = []
        groups[key].append(value)
    return groups.items()


def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]


# Map phase
map_output = list(flatten(map(lambda item: list(mapper(item[0], item[1])), record_reader())))

# Shuffle phase
shuffle_output = list(group_by_key(map_output))

# Reduce phase
reduce_output = list(flatten(map(lambda item: list(reducer(item[0], item[1])), shuffle_output)))

# Create result matrix
result_matrix = np.zeros((I, K))
for (i, k), v3 in reduce_output:
    result_matrix[i, k] = v3

print(result_matrix)

[[1.04843592 0.65513465 1.39534835 0.97491968 0.84008188 1.63965381
  0.93735901 1.17113676 0.98452649 1.25596932 1.17075138 1.61672209
  1.38389525 1.24765548 1.24764461 1.34333702 0.71723039 1.21104307
  1.16266156 1.02402216 0.33484063 0.85142617 1.58648721 0.85627315
  1.1399746  1.27003182 1.17958037 0.95317497 0.8701307  0.52890654
  0.98637407 0.94712714 0.59331385 0.74190439 0.77863214 0.90542808
  0.85164245 0.53733918 0.83860983 0.73334779]
 [1.07713932 0.85797499 1.49469878 1.13573797 1.13314205 1.8433708
  0.84395318 1.11623394 0.91920613 1.46912644 1.43284925 1.75522412
  1.57143638 1.38277145 1.40002174 1.46150327 0.66563041 1.29872354
  1.4623777  1.20279783 0.41266769 1.12129568 1.80884077 1.11033576
  1.12778723 1.35941673 1.15874172 0.90554975 0.99942787 0.61726626
  0.9666419  1.21671171 0.74157428 0.79980434 1.04237413 1.21365559
  1.02893525 0.60203712 0.77876938 0.99899829]]


Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(record_reader, mapper, reducer)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(record_reader, mapper, reducer))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
import numpy as np

I = 2
J = 3
K = 2 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def record_reader():
    """Генерация пар элементов матриц для MapReduce."""
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

def mapper(element1, element2):
    """Функция Map: перемножение элементов и генерация пар (ключ, значение)."""
    (i, j), v1 = element1
    (j, k), v2 = element2
    yield ((i, k), v1 * v2)

def reducer(key, values):
    """Функция Reduce: суммирование произведений для получения элемента матрицы."""
    i, k = key
    yield ((i, k), sum(values))  # Используем встроенную функцию sum()

def group_by_key(items):
    """Группировка элементов по ключу."""
    groups = {}
    for key, value in items:
        if key not in groups:
            groups[key] = []
        groups[key].append(value)
    return groups.items()

def flatten(list_of_lists):
    """Функция для "расплющивания" списка списков."""
    return [item for sublist in list_of_lists for item in sublist]

def map_reduce():
    """Функция для запуска MapReduce."""
    # Map phase
    map_output = list(flatten(map(lambda item: list(mapper(item[0], item[1])), record_reader())))

    # Shuffle phase
    shuffle_output = list(group_by_key(map_output))

    # Reduce phase
    reduce_output = list(flatten(map(lambda item: list(reducer(item[0], item[1])), shuffle_output)))

    # Создание результирующей матрицы
    result_matrix = np.zeros((I, K))
    for (i, k), v3 in reduce_output:
        result_matrix[i, k] = v3
    return result_matrix

# Запуск MapReduce и вывод результата
result_matrix = map_reduce()
print(result_matrix)

[[0.25526613 0.54408154 0.26173409 0.68483773 0.63372258 0.70796781
  0.46867499 0.64258523 0.12464453 0.24662603 0.64201074 0.7371212
  0.10063331 0.12302721 0.44093657 0.09719575 0.05680841 0.79239238
  0.63163864 0.28155107]
 [0.3220669  0.83202495 0.37602716 0.56146246 0.82135845 0.64421977
  0.50256695 0.45525128 0.15421761 0.40358992 0.44488703 0.70641798
  0.22854104 0.27533156 0.70761935 0.10700476 0.08294573 0.62730839
  0.59699153 0.68372867]]


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
maps = 2
reducers = 2

def input_format():
    """Генерация данных для MapReduce с разделением на узлы."""
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield record_reader(range(i, min(i + split_size, I)))  # Ограничиваем верхнюю границу

def record_reader(i_range):
    """Генерация пар элементов матриц для MapReduce."""
    for i in i_range:
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

def mapper(element1, element2):
    """Функция Map: перемножение элементов и генерация пар (ключ, значение)."""
    (i, j), v1 = element1
    (j, k), v2 = element2
    yield ((i, k), v1 * v2)

def reducer(key, values):
    """Функция Reduce: суммирование произведений для получения элемента матрицы."""
    i, k = key
    yield ((i, k), sum(values))  # Используем встроенную функцию sum()

def group_by_key(items):
    """Группировка элементов по ключу."""
    groups = {}
    for key, value in items:
        if key not in groups:
            groups[key] = []
        groups[key].append(value)
    return groups.items()

def flatten(list_of_lists):
    """Функция для "расплющивания" списка списков."""
    return [item for sublist in list_of_lists for item in sublist]

def map_reduce_distributed(input_format_func, map_func, reduce_func, combiner_func=None):
    """Функция для запуска распределенного MapReduce."""
    partitioned_output = []
    for partition_id, record_reader_instance in enumerate(input_format_func()):
        map_output = list(flatten(map(lambda item: list(map_func(item[0], item[1])), record_reader_instance)))

        if combiner_func:
            shuffle_output = list(group_by_key(map_output))
            combined_output = list(flatten(map(lambda item: list(combiner_func(item[0], item[1])), shuffle_output)))
            partitioned_output.append((partition_id, combined_output))
        else:
            partitioned_output.append((partition_id, map_output))  # Не используем combiner

    # Shuffle phase (имитация для распределенной обработки)
    all_map_output = list(flatten([partition[1] for partition in partitioned_output]))
    shuffle_output = list(group_by_key(all_map_output))

    reduce_output = list(flatten(map(lambda item: list(reduce_func(item[0], item[1])), shuffle_output)))

    return reduce_output


# Запуск MapReduce и вывод результата
partitioned_output = map_reduce_distributed(input_format, mapper, reducer)

result_matrix = np.zeros((I, K))
for (i, k), v3 in partitioned_output:
    result_matrix[i, k] = v3

print(result_matrix)
# np.allclose(reference_solution, result_matrix)  # Раскомментируйте для проверки, если reference_solution определен

[[1.68558829 1.18988123 1.47123115 1.15372909 1.15937793 1.42268992
  0.69690222 1.10304897 1.34331761 0.52967067 1.36110252 1.20571905
  1.1419233  0.93510027 1.67862853 0.93282339 0.95870768 1.11804185
  0.93797935 0.89218879 1.53472514 1.41268974 1.2429247  1.21425579
  0.49787852 1.1059188  1.41549145 1.21638405 1.23424365 0.36708492
  1.43771313 0.92303833 1.91821538 1.31101617 0.89938842 0.90007177
  1.09607282 0.80995146 1.25166675 0.87199719]
 [0.94316092 0.55881671 0.64798935 0.50954323 0.70019891 0.69003
  0.35622557 0.77645309 0.79828071 0.34912151 0.81070049 0.77554901
  0.51957326 0.69063495 0.77497784 0.35049636 0.37051803 0.71797422
  0.39990921 0.2966627  0.6155372  0.84347311 0.78133495 0.70347931
  0.31090984 0.69362843 0.75637434 0.7676824  0.48323669 0.12877851
  0.78591622 0.53097219 1.00173728 0.43163854 0.4359571  0.61270758
  0.3502672  0.38383032 0.69961422 0.51192258]]


Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
reference_solution = np.matmul(small_mat, big_mat)
maps = 3
reducers = 2


def input_format():
    """Генерация данных для MapReduce с разделением на узлы."""
    first_mat = []
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            first_mat.append(((0, i, j), small_mat[i, j]))  # Первая матрица

    split_size = int(np.ceil(len(first_mat) / maps))
    for i in range(0, len(first_mat), split_size):
        yield first_mat[i:i + split_size]

    second_mat = []
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            second_mat.append(((1, j, k), big_mat[j, k]))  # Вторая матрица

    split_size = int(np.ceil(len(second_mat) / maps))
    for i in range(0, len(second_mat), split_size):
        yield second_mat[i:i + split_size]


def map_join(k1, v1):
    """Функция Map для соединения матриц."""
    mat_num, i, j = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))
    else:
        yield (i, (mat_num, j, w))


def reduce_join(key, values):
    """Функция Reduce для соединения матриц."""
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])


def map_mul(k1, v1):
    """Функция Map для умножения."""
    yield (k1, v1)


def reduce_mul(key, values):
    """Функция Reduce для умножения."""
    yield (key, sum(values))


def group_by_key(items):
    """Группировка элементов по ключу."""
    groups = {}
    for key, value in items:
        if key not in groups:
            groups[key] = []
        groups[key].append(value)
    return groups.items()


def flatten(list_of_lists):
    """Функция для "расплющивания" списка списков."""
    return [item for sublist in list_of_lists for item in sublist]


def map_reduce_distributed(input_format_func, map_func, reduce_func, combiner_func=None):
    """Функция для запуска распределенного MapReduce."""
    partitioned_output = []
    for partition_id, data_partition in enumerate(input_format_func()):
        map_output = list(flatten(map(lambda item: list(map_func(item[0], item[1])), data_partition)))

        if combiner_func:
            shuffle_output = list(group_by_key(map_output))
            combined_output = list(flatten(map(lambda item: list(combiner_func(item[0], item[1])), shuffle_output)))
            partitioned_output.append((partition_id, combined_output))
        else:
            partitioned_output.append((partition_id, map_output))

    # Shuffle phase (имитация для распределенной обработки)
    all_map_output = list(flatten([partition[1] for partition in partitioned_output]))
    shuffle_output = list(group_by_key(all_map_output))

    reduce_output = list(flatten(map(lambda item: list(reduce_func(item[0], item[1])), shuffle_output)))

    return reduce_output


# Запуск MapReduce и вывод результата
joined_data = map_reduce_distributed(input_format, map_join, reduce_join)
mul_output = map_reduce_distributed(lambda: [joined_data], map_mul, reduce_mul) # Передаем joined_data как входные данные

result = {}
for k, v in mul_output:
    result[k] = v

result_matrix = np.zeros((I, K))
for (i, k), v in result.items():
    result_matrix[i, k] = v

print(result_matrix)
print(np.allclose(reference_solution, result_matrix))

[[1.20928633 1.48913242 1.01647125 0.54272289 0.35472154 0.42960992
  0.67871319 0.74970423 1.19475358 0.54879002 0.41721122 0.68859295
  1.29479623 0.26887045 0.41914725 0.85973628 0.86039432 0.66151178
  1.33251781 0.96466454 1.41076858 1.15445722 0.75228948 0.34249112
  0.78348978 1.4819058  1.10261813 0.23037401 0.55177301 1.34699128
  0.83621013 1.22909298 0.42935773 1.05944498 0.7789716  0.96372675
  0.85899563 0.51371641 0.74555587 0.80626904]
 [0.60556172 0.9232288  0.6858455  0.57615961 0.3875836  0.26883014
  0.36355993 0.6385571  0.64375302 0.19683835 0.37364157 0.63012623
  0.83554501 0.11784201 0.49987115 0.71561548 0.33621963 0.58211259
  0.90569154 0.86515976 0.7656528  0.63287319 0.86149852 0.64207519
  0.32992228 0.96957608 0.88552963 0.31471394 0.47666661 0.86604119
  0.49669243 0.60386743 0.20071638 0.54032538 0.54365326 0.33208972
  0.52929942 0.35337076 0.51207101 0.57506565]]
True
